# Project 3

In this project, you will perform a logistic regression on the admissions data we've been working with in projects 1 and 2.

In [1]:
%matplotlib inline
import matplotlib.pyplot as plt
import pandas as pd
import statsmodels.api as sm
import pylab as pl
import numpy as np


In [3]:
df_raw = pd.read_csv("../assets/admissions.csv")
df = df_raw.dropna() 
print df.head()

   admit    gre   gpa  prestige
0      0  380.0  3.61       3.0
1      1  660.0  3.67       3.0
2      1  800.0  4.00       1.0
3      1  640.0  3.19       4.0
4      0  520.0  2.93       4.0


## Part 1. Frequency Tables

#### 1. Let's create a frequency table of our variables

In [16]:
# frequency table for prestige and whether or not someone was admitted
#df.groupby('prestige').count()
df['prestige'].value_counts()

2.0    148
3.0    121
4.0     67
1.0     61
Name: prestige, dtype: int64

## Part 2. Return of dummy variables

#### 2.1 Create class or dummy variables for prestige 

In [22]:
dummy_ranks = pd.get_dummies(df.prestige, prefix='prestige').iloc[:, :3]

# concatenate the dummy variable columns onto the original DataFrame (axis=0 means rows, axis=1 means columns)
data = pd.concat([df[['admit', 'gre','gpa']], dummy_ranks], axis=1)
#data.head()
data_df = data.rename(columns={'prestige_1.0': 'prestige_1','prestige_2.0': 'prestige_2', 'prestige_3.0': 'prestige_3'})
data_df

,admit,gre,gpa,prestige_1,prestige_2,prestige_3
0,0,380.0,3.61,0.0,0.0,1.0
1,1,660.0,3.67,0.0,0.0,1.0
2,1,800.0,4.00,1.0,0.0,0.0
3,1,640.0,3.19,0.0,0.0,0.0
4,0,520.0,2.93,0.0,0.0,0.0
5,1,760.0,3.00,0.0,1.0,0.0
6,1,560.0,2.98,1.0,0.0,0.0
7,0,400.0,3.08,0.0,1.0,0.0
8,1,540.0,3.39,0.0,0.0,1.0
9,0,700.0,3.92,0.0,1.0,0.0


#### 2.2 When modeling our class variables, how many do we need? 



Answer: 3. As the there are 4 possible values but the remaining can be derived based on the values of 3.

## Part 3. Hand calculating odds ratios

Develop your intuition about expected outcomes by hand calculating odds ratios.

In [23]:
cols_to_keep = ['admit', 'gre', 'gpa']
handCalc = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_1':])
print handCalc.head()

   admit    gre   gpa  prestige_1.0  prestige_2.0  prestige_3.0
0      0  380.0  3.61           0.0           0.0           1.0
1      1  660.0  3.67           0.0           0.0           1.0
2      1  800.0  4.00           1.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           0.0
4      0  520.0  2.93           0.0           0.0           0.0


In [45]:
#crosstab prestige 1 admission 
# frequency table cutting prestige and whether or not someone was admitted
#handCalc['prestige_1.0'].value_counts()
pd.crosstab(handCalc.admit,handCalc['prestige_1.0'])

prestige_1.0,0.0,1.0
admit,,
0,243,28
1,93,33


#### 3.1 Use the cross tab above to calculate the odds of being admitted to grad school if you attended a #1 ranked college

In [86]:
od_p1=33.0/28
od_p1

1.1785714285714286

#### 3.2 Now calculate the odds of admission if you did not attend a #1 ranked college

In [87]:
od_p0=93.0/243
od_p0

0.38271604938271603

#### 3.3 Calculate the odds ratio

In [91]:
#1.1785714285714286/0.38271604938271603
or_p=od_p1/od_p0
or_p

3.079493087557604

#### 3.4 Write this finding in a sentenance: 

Answer: The Odds raatio of getting admition give that you attend the No 1 school is high. Meaning that the odds of getting admission is 3 times higher if you attended the prestige 1 school.

#### 3.5 Print the cross tab for prestige_4

In [72]:
#handCalc.head()
#(handCalc['prestige_1.0'] + handCalc['prestige_2.0'] + handCalc['prestige_3.0'] <1).astype(int)
pd.crosstab(handCalc.admit,(handCalc['prestige_1.0'] + handCalc['prestige_2.0'] + handCalc['prestige_3.0'] <1).astype(int))


col_0,0,1
admit,,
0,216,55
1,114,12


#### 3.6 Calculate the OR 

In [96]:
od1=12.0/55
od1
od2=114.0/216
od2
or2=od1/od2
or2

0.4133971291866028

#### 3.7 Write this finding in a sentence

Answer: The Odds raatio of getting admition give that you attend the school of prestige 4 is low. Meaning that the odds of getting admission is .5 times lower if you attended the prestige 4 school.

## Part 4. Analysis

In [111]:
# create a clean data frame for the regression
cols_to_keep = ['admit', 'gre', 'gpa']
dummy_ranks = pd.get_dummies(df.prestige, prefix='prestige').iloc[:, 1:]
data = df[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])
print data.head()
#dummy_ranks

   admit    gre   gpa  prestige_2.0  prestige_3.0  prestige_4.0
0      0  380.0  3.61           0.0           1.0           0.0
1      1  660.0  3.67           0.0           1.0           0.0
2      1  800.0  4.00           0.0           0.0           0.0
3      1  640.0  3.19           0.0           0.0           1.0
4      0  520.0  2.93           0.0           0.0           1.0


We're going to add a constant term for our Logistic Regression. The statsmodels function we're going to be using requires that intercepts/constants are specified explicitly.

In [112]:
# manually add the intercept
data['intercept'] = 1.0

#### 4.1 Set the covariates to a variable called train_cols

In [113]:
train_cols=data.columns[1:]
train_cols

Index([u'gre', u'gpa', u'prestige_2.0', u'prestige_3.0', u'prestige_4.0',
       u'intercept'],
      dtype='object')

#### 4.2 Fit the model

In [115]:
logit = sm.Logit(data['admit'], data[train_cols])

# fit the model
result = logit.fit()

Optimization terminated successfully.
         Current function value: 0.573854
         Iterations 6


#### 4.3 Print the summary results

In [116]:
print result.summary()

                           Logit Regression Results                           
Dep. Variable:                  admit   No. Observations:                  397
Model:                          Logit   Df Residuals:                      391
Method:                           MLE   Df Model:                            5
Date:                Sat, 10 Dec 2016   Pseudo R-squ.:                 0.08166
Time:                        22:47:33   Log-Likelihood:                -227.82
converged:                       True   LL-Null:                       -248.08
                                        LLR p-value:                 1.176e-07
                   coef    std err          z      P>|z|      [95.0% Conf. Int.]
--------------------------------------------------------------------------------
gre              0.0022      0.001      2.028      0.043      7.44e-05     0.004
gpa              0.7793      0.333      2.344      0.019         0.128     1.431
prestige_2.0    -0.6801      0.317     -2.14

#### 4.4 Calculate the odds ratios of the coeffiencents and their 95% CI intervals

hint 1: np.exp(X)

hint 2: conf['OR'] = params
        
           conf.columns = ['2.5%', '97.5%', 'OR']

In [127]:
#result.params
# Odds Ratio
np.exp(result.params)


gre             1.002221
gpa             2.180027
prestige_2.0    0.506548
prestige_3.0    0.262192
prestige_4.0    0.211525
intercept       0.020716
dtype: float64

In [129]:
#conf.columns = ['2.5%', '97.5%', 'OR']
param_val = result.params
conf = result.conf_int()
conf['OR'] = param_val
conf.columns = ['2.5%', '97.5%', 'OR']
print np.exp(conf)

                  2.5%     97.5%        OR
gre           1.000074  1.004372  1.002221
gpa           1.136120  4.183113  2.180027
prestige_2.0  0.272168  0.942767  0.506548
prestige_3.0  0.133377  0.515419  0.262192
prestige_4.0  0.093329  0.479411  0.211525
intercept     0.002207  0.194440  0.020716


#### 4.5 Interpret the OR of Prestige_2

Answer: Chances of getting admitted for a student who attended a prestige 2 school are half( or 0.5 times) less then others.

#### 4.6 Interpret the OR of GPA

Answer: Based on gpa alone, the chances of getting admitted are twice likely

## Part 5: Predicted probablities


As a way of evaluating our classifier, we're going to recreate the dataset with every logical combination of input values. This will allow us to see how the predicted probability of admission increases/decreases across different variables. First we're going to generate the combinations using a helper function called cartesian (above).

We're going to use np.linspace to create a range of values for "gre" and "gpa". This creates a range of linearly spaced values from a specified min and maximum value--in our case just the min/max observed values.

In [147]:
def cartesian(arrays, out=None):
    """
    Generate a cartesian product of input arrays.
    Parameters
    ----------
    arrays : list of array-like
        1-D arrays to form the cartesian product of.
    out : ndarray
        Array to place the cartesian product in.
    Returns
    -------
    out : ndarray
        2-D array of shape (M, len(arrays)) containing cartesian products
        formed of input arrays.
    Examples
    --------
    >>> cartesian(([1, 2, 3], [4, 5], [6, 7]))
    array([[1, 4, 6],
           [1, 4, 7],
           [1, 5, 6],
           [1, 5, 7],
           [2, 4, 6],
           [2, 4, 7],
           [2, 5, 6],
           [2, 5, 7],
           [3, 4, 6],
           [3, 4, 7],
           [3, 5, 6],
           [3, 5, 7]])
    """

    arrays = [np.asarray(x) for x in arrays]
    dtype = arrays[0].dtype

    n = np.prod([x.size for x in arrays])
    if out is None:
        out = np.zeros([n, len(arrays)], dtype=dtype)

    m = n / arrays[0].size
    out[:,0] = np.repeat(arrays[0], m)
    if arrays[1:]:
        cartesian(arrays[1:], out=out[0:m,1:])
        for j in xrange(1, arrays[0].size):
            out[j*m:(j+1)*m,1:] = out[0:m,1:]
    return out

In [148]:
# instead of generating all possible values of GRE and GPA, we're going
# to use an evenly spaced range of 10 values from the min to the max 
gres = np.linspace(data['gre'].min(), data['gre'].max(), 10)
print gres
# array([ 220.        ,  284.44444444,  348.88888889,  413.33333333,
#         477.77777778,  542.22222222,  606.66666667,  671.11111111,
#         735.55555556,  800.        ])
gpas = np.linspace(data['gpa'].min(), data['gpa'].max(), 10)
print gpas
# array([ 2.26      ,  2.45333333,  2.64666667,  2.84      ,  3.03333333,
#         3.22666667,  3.42      ,  3.61333333,  3.80666667,  4.        ])


# enumerate all possibilities
combos = pd.DataFrame(cartesian([gres, gpas, [1, 2, 3, 4], [1.]]))

[ 220.          284.44444444  348.88888889  413.33333333  477.77777778
  542.22222222  606.66666667  671.11111111  735.55555556  800.        ]
[ 2.26        2.45333333  2.64666667  2.84        3.03333333  3.22666667
  3.42        3.61333333  3.80666667  4.        ]


#### 5.1 Recreate the dummy variables

In [149]:
# recreate the dummy variables
combos.columns = ['gre', 'gpa', 'prestige', 'intercept']
dummy_ranks = pd.get_dummies(combos['prestige'], prefix='prestige')
dummy_ranks.columns = ['prestige_1', 'prestige_2', 'prestige_3', 'prestige_4']

# keep only what we need for making predictions
cols_to_keep = ['gre', 'gpa', 'prestige', 'intercept']
combos = combos[cols_to_keep].join(dummy_ranks.ix[:, 'prestige_2':])


#### 5.2 Make predictions on the enumerated dataset

In [154]:
#combos
train_cols=data.columns[1:]
train_cols
combos['admit_pred'] = result.predict(combos[train_cols])
#print combos.head()

KeyError: "Index([u'prestige_2.0', u'prestige_3.0', u'prestige_4.0'], dtype='object') not in index"

#### 5.3 Interpret findings for the last 4 observations

Answer: 

## Bonus

Plot the probability of being admitted into graduate school, stratified by GPA and GRE score.